# Developing the LLM Tools python module

> The code developed here is finally copied to `llm/tool.py` and available as the `llm.tool` module

This notebook will detail the creation and evolution of the `llm.tools` module which will contain the definition of what a LLM `Tool` is_(as introduced by OpenAI)_. Parallelly, the driver of the chat _(which uses said tools)_ is being documented at [Py_mod_llm_controller_tools_devel.ipynb](./Py_mod_llm_controller_tools_devel.ipynb). 

This notebook develops the tool-calling functionality on top of plain one-shot prompting. While OpenAI is chosen as the LLM provider, the code should work for most everyone with hopefully minor modifcations.

While `tool` and `using tools in LLM interactions` belong together, I am splitting the development into two notebooks since a single notbook covering both aspects was getting too large.
 - This notebook [Py_mod_llm_tools_devel.ipynb](./Py_mod_llm_tools_devel.ipynb) helps define the tools, generating their schemas and such.
 - [Py_mod_llm_controller_tools_devel.ipynb](./Py_mod_llm_controller_tools_devel.ipynb) develops the higher-level loop/driver that takes in a bunch of tool definitions and drives a chat to completion.

 ![](../img/tool_calling_protocol.png)

The hello-world of tool-calling, _get_weather_ is described in the official [OpenAI docs](https://platform.openai.com/docs/guides/function-calling). The OpenAI implementation _(atleast when it came out in late 2023)_ did not specify how one would build the json spec.Our implementation here demonstrates the use of pydantic classes to
 - Automate generation of the JSON schema required for tools
 - Automate deserialization of the JSON args supplied by OpenAI
 - Simplify implementation of tooling for ReAct and any other LLM use case.
 - Similar infra is also used for obtaining structured outputs from OpenAI and others.
 - Additionally, we also build a way to use regular python functions _(plain python parameter types)_ to make everything super-easy like LangChain/LlamaIndex.

![](../img/tool_calling_protocol_impl_details.png)

## Notebook Setup

In [1]:
# Allow Colab or VSCore/Normal Jupyter environments
import os
from pathlib import Path

# The default relative path when running the notebook from a cloned repo.
LIB_PATH = Path("../lib")

if 'google.colab' in str(get_ipython()):
    print("👉 Setting up for Colab")
    # This will create a py-llm dir at the same level as this notebook
    # Refer to the lib in there using `./py-llm/lib` as opposed to the 
    # relative `../lib` when we are running straight from the py-llm/nbs 
    # directory in VScode.
    if not os.path.isdir("./py-llm"):
        print("Cloning git repo into ./py-llm")
        !git clone -b 3_llm_tools_and_support https://github.com/vamsi-juvvi/py-llm.git
        LIB_PATH = Path("./py-llm/lib")
    else:
        print("./py-llm exists. Not cloning") 

In [9]:
# Append to sys.path directly
# Make sure to `str(Path)`
# - The resolve() converts relative to absolute. 
# - If you use ~ for HOME, use `Path.expand_user()`
import sys
import logging

sys.path.append(str(LIB_PATH.resolve()))

from py_llm.util import jupyter_util
from py_llm.util.jupyter_util import DisplayHTML as DH
from py_llm.util.jupyter_util import DisplayMarkdown as DM
from py_llm.util.jupyter_util import ColabEnv

# Init logging at DEBUG. Once we are done testing, this can drop 
# down to warning
jupyter_util.setup_logging(logging.INFO)

In [3]:
# Uncomment for use in Colab or when the package is missing
#!pip install -qy openai
#!pip install -qy jsonref

In [4]:
import os
import openai

# If you want to log OpenAI's python library itself, also set the log level for this
# normally, limit this to warning/error and keep your own logging at debug levels.
# If this doesn't work right away, restart the kernel after changing the log-level
os.environ["OPENAI_LOG"]="error"

# Finally ensure you have the OpenAI key.
openai.api_key = ColabEnv.colab_keyval_or_env("OPENAI_API_KEY")
assert(openai.api_key)

# Final API

## llm.tools module

Tools are central to all advanced LLM use cases. I started out with raw tools _(plain json processing)_ and slowly ended up at the version where a tool is simply a python function with a json-serializable (pydantic) argument. 

In [5]:
#------------------------------------------------------------------------
# Tool execution infrastructure
#------------------------------------------------------------------------
import inspect
import jsonref
from typing import TypeVar
import json
from pydantic import ValidationError, BaseModel, create_model


class InPromptToolSchema:    
    """
    Holds the definition of the in-prompt schema and a collection
    of methods for fixed string generation recipes.
    """
    def __init__(self, name:str, desc:str, arg_json_str:str):
        self.name = name
        self.description = desc
        self.arg_json_str = arg_json_str

    # Follows the LlamaIndex format.
    # Use the fields directly for custom formats.
    def __str__(self):
        _fields = [f"Tool Name : {self.name}"]

        if self.description : 
            _fields.append(f"Tool Description : {self.description}")

        if self.arg_json_str: 
            _fields.append(f"Tool Args: {self.arg_json_str}")
        else:
            _fields.append(f"Tool Args: tool takes no arguments, use an empty string \"\" ")

        return "\n".join(_fields)            

F = TypeVar('F')

class Tool:    
    # tool_fn: fn(PyDanticObject) -> str
    def __init__(self,               
                 tool_fn):
        
        logging.debug(f"Tool : {tool_fn.__name__}, Initialization")
        _items = Tool.build_tool_call_items(tool_fn)
        assert(len(_items) == 3)

        # Name of the tool taken from function name.
        self.name                  = tool_fn.__name__

        # Tool Schema suitable for OpenAI tool calls. 
        # `tools=[schema1, schema2,..]``
        self.tool_schema           = _items[0]        
        logging.debug(f"Tool : {tool_fn.__name__}, Schema=\n{self.tool_schema}\n")

        # Lambd to deserialize from string
        self.tool_arg_deserializer = _items[1] 

        # lambda to call function with deserialized arg (if any) 
        # or empty arg-list
        self.tool_func             = _items[2]

        # Tool schema suitable for in-prompt use (less verboce 
        # than full schema)
        # Type: InPromptToolSchema, not JSON
        self.in_prompt_schema      = Tool.build_inprompt_tool_schema(self.tool_schema)       
        logging.debug(f"Tool : {tool_fn.__name__}, InPromptSchema=\n{self.in_prompt_schema}\n") 
    
    def exec(self, json_arg: str) -> str:
        if self.tool_arg_deserializer:
            try:
                logging.debug(f"Attempting to deserialize {json_arg} for tool: {self.name}")

                arg_obj = self.tool_arg_deserializer(json_arg)
                logging.debug(f"✔️ deserialized to {arg_obj}. Calling function")

                func_result = self.tool_func(arg_obj)
                logging.debug(f"✔️ function returned {func_result}")

                return func_result

            except ValidationError as e:
                logging.error(f"JSON string {json_arg} is not valid for {self.name}:{e}")
        else:            
            logging.debug(f"Calling no-arg function: {self.name}")            
            assert(json_arg == '{}' or json_arg == "" or json_arg is None)
            
            func_result = self.tool_func()
            logging.debug(f"✔️ function returned {func_result}")
            return func_result
    
    @staticmethod
    def build_inprompt_tool_schema(tool_schema:json) -> InPromptToolSchema:
        """
        Given a full tool-schema (as generated by build_tool_call_items), this builds 
        a simplified function schema meant for insertion into a prompt.

        Meant for use by the ctor. Not directly by client code except for testing.
        """    
        func_schema = tool_schema["function"]

        return InPromptToolSchema(
            name = func_schema["name"],
            desc = func_schema["description"] if "description" in func_schema else None,
            arg_json_str = json.dumps(func_schema["parameters"]) if "parameters" in func_schema else None
        )

    @staticmethod
    def build_tool_call_items(f:F):
        """
        f: A function of the form 'func(arg:BaseModel)` | `func()`
        Max of 1 argument and it should be a class deriving from PyDantic BaseModel  

        Returns a tuple: (tool_schema, arg_schema, arg_deserializer, func_call)
            tool_schema      - JSON representing the function/tool schema required by OpenAI etc
            arg_schema       - JSON representing the arg portion of the schema required by OpenAI etc
                               Meant for use inside prompts (ReAct prompts)
            arg_deserializer - f(str) -> PyDanticObj. Lambda to deserialize from string to the 
                               arg type used by the function
            func_call        - f(obj) -> str. Lambda to invoke the function with the deserialized json

        Eg.

        @dataclass
        class GetWeather(BaseModel):        
            location : str = Field(description="City and country e.g. San Jose, USA")

        def get_weather(args: GetWeather) -> float:
            '''
            Get current temperature for a given location.
            '''
            return 10    

        And use this thus:
        
            (tool_schema, arg_schema, arg_deserializer, fn_call) = build_tool_call_items(get_weather)
        """    
        if not inspect.isfunction(f):
            raise TypeError(f"{str(f)} should be a python function")

        sig = inspect.signature(f)
        num_params = len(sig.parameters)

        if num_params == 0:
            # 0 args case            
            return Tool._build_tool_call_items_0_args(f)

        else:            
            params = list(sig.parameters.items())
            p_ann   = params[0][1].annotation            
            p_class = p_ann.__class__

            # Special: 1 arg case with BaseModel argument. We directly use it's serialization
            # Special power to add docs per-field etc.
            if num_params == 1 and "pydantic._internal._model_construction.ModelMetaclass" in str(p_class):
                return Tool._build_tool_call_items_1_arg_pydantic(f) 
            else:
                # Dynamically create the BaseModel 
                # and build the rest from that                
                return Tool._build_tool_call_items_n_arg_dynamic(f)
    
    def _build_tool_call_items_0_args(fn):
        """
        Helper for build_tool_call_items
        """
        # See https://platform.openai.com/docs/guides/function-calling
        # Note: OpenAI does not like it if we send a parameters key with nul
        #       so not using that key at all.
        tool_schema = Tool._build_tool_schema_json(fn, None)        
        func_arg_deserializer = None
        func_call             = lambda : fn()        

        return (tool_schema, func_arg_deserializer, func_call)
    
    def _build_tool_call_items_1_arg_pydantic(fn):
        """
        Helper for build_tool_call_items when there is 1 arg 
        and it is a Pydantic BaseModel
        """
        sig = inspect.signature(fn)
        assert(len(sig.parameters) == 1)
        
        params = list(sig.parameters.items())
        p_ann   = params[0][1].annotation            
        p_class = p_ann.__class__
        assert("pydantic._internal._model_construction.ModelMetaclass" in str(p_class))

        # exec the class method to generate the json schema
        arg_schema = getattr(p_ann, 'model_json_schema').__call__()
        arg_schema = jsonref.replace_refs(arg_schema)

        # 👉 OpenAI enforces that "additionalProperties" : False is set!
        arg_schema["additionalProperties"] = False
                    
        # Now generate the schema for the full function
        tool_schema = Tool._build_tool_schema_json(fn, arg_schema)        

        # generate lambda to deserialize from stringified json
        # use pydantic's model_validate_json method
        des_callable = getattr(p_ann, 'model_validate_json')
        func_arg_deserializer = lambda json_str: des_callable.__call__(json_str)

        # generate lambda to call the function with deaserialized json
        func_call = lambda deserialized: fn(deserialized)   

        return (tool_schema, func_arg_deserializer, func_call)
    
    def _build_tool_call_items_n_arg_dynamic(fn):
        """
        Helper for build_tool_call_items when there are more than one
        args of any type. 

        a Pydantic BaseModel is built dynamically from the signature
        """
        sig = inspect.signature(fn)
        assert(len(sig.parameters))

        # Build args to send to pydantic.create_model
        dyn_create_model_args = [f"\"{fn.__name__}_args\""]
        call_args_vec  = []
        for p in sig.parameters.values():    
            # build arg to generate the model
            arg = f"{p.name}=({p.annotation.__name__}, ...)"                
            dyn_create_model_args.append(arg)    

            # build arg to call fn using model fields
            # Assumes deserialized model-object will be called `des_obj`
            # (deserialized object)
            call_arg = f"{p.name} = des_obj.{p.name}"
            call_args_vec.append(call_arg)            

        cmd = f"create_model({", ".join(dyn_create_model_args)})"        
        dyn_model = eval(cmd)

        # Schema ----------
        arg_schema = dyn_model.model_json_schema()
        arg_schema = jsonref.replace_refs(arg_schema)                
        arg_schema["additionalProperties"] = False   # 👉 OpenAI needs this.

        # Now generate the schema for the full function
        # See https://platform.openai.com/docs/guides/function-calling
        tool_schema = Tool._build_tool_schema_json(fn, arg_schema)

        # deserializer ---
        func_arg_deserializer = lambda jstr: dyn_model.model_validate_json(jstr)
        
        # caller ---------
        # This is a bit complex. What I want for `func_call` is 
        # lambda deserialized_object : fn(deserialized_object)
        # However, when this is built using eval, it does not close over fn!
        # only globals. So we need a second level of indirection with a function 
        # that does close over fn
        call_eval_str = f"lambda des_obj, fn: fn({", ".join(call_args_vec)})"
        logging.info(f"eval string for func calling = {call_eval_str}")
        call_fn_2 = eval(call_eval_str)                   # does not close over fn, so sending fn as arg
        func_call= lambda des_obj: call_fn_2(des_obj, fn) # this closes over fn and can call call_fn_2

        return (tool_schema, func_arg_deserializer, func_call)
    
    def _build_tool_schema_json(fn, arg_schema):
        # See https://platform.openai.com/docs/guides/function-calling
        tool_schema = {
            "type"       : "function",
            "function"   : {
                "name"       : fn.__name__,
                "description": fn.__doc__.strip() if fn.__doc__ else f"function {fn.__name__}",                
                "strict"     : True
            }
        }            

        if arg_schema:
            tool_schema["function"]["parameters"] = arg_schema
        else:
            # empty args is not accepted. Seems to vary by mood of OpenAI
            # this works
            # 👉 OpenAI enforces that "additionalProperties" : False is set!
            tool_schema["function"]["parameters"] = {
                "type" : "object",
                "properties" : {},
                "required"   : [],
                "additionalProperties" : False
            }

        return tool_schema

        
                    
#------------------------------------------------------------------------------
class ToolCollection:
    def __init__(self, tools= None):
        """
        tools is a list of Tool or callable
        """
        # {name : Tool}
        self.tool_dict = {}

        if tools:
            for t in tools:
                self.register_tool(t)

    def register_tool(self, tool):
        """
         Register's a tool by name for later us
         tool : Tool | python function.
        """
        if inspect.isfunction(tool):            
            tool = Tool(tool)

        if tool.name in self.tool_dict:
            logging.warn(f"Tool {tool.name} has already been registed. Overwriting!")
        
        assert(tool.tool_func)
        assert(tool.tool_schema)
        self.tool_dict[tool.name] = tool
        
    def exec_tool(self, name: str, args: str) -> str :
        if not name in self.tool_dict:
            raise KeyError(f"Tool: {name} is not registered! Cannot call!")
        else:            
            tool = self.tool_dict[name]
            logging.debug(f"Executing tool: {tool.name}")
            return tool.exec(args)
        
    def get_tool_names(self):
        return [key for key in self.tool_dict.keys()]
    
    def get_schemas(self, mapper=None) :
        mapper = mapper if mapper else lambda x: x
        return [mapper(tool.tool_schema) for tool in self.tool_dict.values()]
    
    def get_inprompt_schemas(self, mapper=None) :
        mapper = mapper if mapper else lambda x: x
        return [mapper(tool.in_prompt_schema) for tool in self.tool_dict.values()]        
    
#-------------------------------------------------------------------------------
# fake module
class tools:
    Tool = Tool
    ToolCollection = ToolCollection

### Test the various tool scenarios

In [11]:
#----------------------------------------------------------------------------
# Test schema generation static methods in Tool
def exercise_tool_methods(test_nm, test_fn, json_str, expect_eval):

    items = Tool.build_tool_call_items(test_fn)
    assert(len(items) == 3)
    (tool_schema, arg_deserializer, fn_caller) = items

    DM.md(f"""
## Testing Tool methods directly - {test_nm}
    
### tool schema output - {test_nm}

{DM.json_fmt(tool_schema)}

### in-prompt schema output - {test_nm}

```
{Tool.build_inprompt_tool_schema(tool_schema)}
```

### Deserialized from string - {test_nm}

Deserializing {json_str}
↪

{DM.code_fmt(arg_deserializer(json_str)) if arg_deserializer else "⚠️ deserializer is NONE"}

### Function called with deserialized - {test_nm}

 - 👉 Expecting **{expect_eval}**
 - 👉 Got **{str(fn_caller(arg_deserializer(json_str))) if arg_deserializer else str(fn_caller())}** 
""")

#----------------------------------------------------------------------------
# Test deserialization and calling via Tool
def exercise_tool_api(test_nm, test_fn, json_str, expect_eval):

        tool = Tool(test_fn)
        DM.md(f"""
----     
## Testing Tool API - {test_nm}
               
### Tool: tool schema output - {test_nm}

{DM.json_fmt(tool.tool_schema)}

### Tool: in-prompt schema output - {test_nm}

```
{tool.in_prompt_schema}
```

### Tool: Function called with deserialized - {test_nm}

 - Deserializing {json_str}
 - Expecting **{expect_eval}**
 - Got ↓
 
```
{str(tool.exec(json_str))}
```
""")

#----------------------------------------------------------------------------
# Test schema_generation and calling via ToolCollection
def exercise_toolcollection_api(test_nm, test_fn, json_str, expect_eval):

     # Register in the tool dictionary
     # Note that one could evolve this further to make the tool_dict the single 
     # source and use it to the tool descriptions as well.
     gw_tools = ToolCollection()
     gw_tools.register_tool(            
         tool=Tool(test_fn)
     )                        

     DM.md(f"""
----     
## Testing ToolCollection API - {test_nm}
           
### ToolCollection: exec with serialized string - {test_nm}

 - Deserializing {json_str} for {test_fn.__name__}
 - **Expecting** {expect_eval}
 - **Got** ↓

```
{gw_tools.exec_tool(test_fn.__name__, json_str)}
```

### ToolCollection: all schemas - {test_nm}

```json
{str(gw_tools.get_inprompt_schemas(mapper=lambda ips: str(ips)))}
```
""")                              
     
#----------------------------------------------------------------------------
# Driver to run all the tests
def exercise_all(test_nm, test_fn, json_str, expect_eval):
     exercise_tool_methods(test_nm, test_fn, json_str, expect_eval)
     exercise_tool_api(test_nm, test_fn, json_str, expect_eval)
     exercise_toolcollection_api(test_nm, test_fn, json_str, expect_eval)    

#### Single arg function with pydantic

This tool form is what I started out with since it was the most general
 - Capture any number of args into a single struct
 - Each field can be assigned a descriptive docstring
 - function itself gets a doc string

The usability though, is a bit poor
 - Forcing the developer to wrap arguments into a Pydantic Basemodel _(even if it is just a few lines) _
 - Not pythonic since it does not allow normal python functions

In [12]:
# Exercise 
# fn(arg) where arg is a Pydantic model
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class GetWeather(BaseModel):        
    location : str = Field(description="City and country e.g. San Jose, USA")

def get_weather(args: GetWeather) -> float:
    """
    Get current temperature for a given location.
    """
    return "10"    

SERIALIZED_DATA_SAMPLE = "{\"location\":\"Paris, France\"}"
EXPECTED = "10"

exercise_all(test_nm="fn(pydantic)",              
             test_fn=get_weather, 
             json_str = SERIALIZED_DATA_SAMPLE, 
             expect_eval=EXPECTED)


## Testing Tool methods directly - fn(pydantic)

### tool schema output - fn(pydantic)

```json
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "strict": true,
        "parameters": {
            "properties": {
                "location": {
                    "description": "City and country e.g. San Jose, USA",
                    "title": "Location",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ],
            "title": "GetWeather",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### in-prompt schema output - fn(pydantic)

```
Tool Name : get_weather
Tool Description : Get current temperature for a given location.
Tool Args: {"properties": {"location": {"description": "City and country e.g. San Jose, USA", "title": "Location", "type": "string"}}, "required": ["location"], "title": "GetWeather", "type": "object", "additionalProperties": false}
```

### Deserialized from string - fn(pydantic)

Deserializing {"location":"Paris, France"}
↪

```
location='Paris, France'
```

### Function called with deserialized - fn(pydantic)

 - 👉 Expecting **10**
 - 👉 Got **10** 



----     
## Testing Tool API - fn(pydantic)

### Tool: tool schema output - fn(pydantic)

```json
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "strict": true,
        "parameters": {
            "properties": {
                "location": {
                    "description": "City and country e.g. San Jose, USA",
                    "title": "Location",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ],
            "title": "GetWeather",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### Tool: in-prompt schema output - fn(pydantic)

```
Tool Name : get_weather
Tool Description : Get current temperature for a given location.
Tool Args: {"properties": {"location": {"description": "City and country e.g. San Jose, USA", "title": "Location", "type": "string"}}, "required": ["location"], "title": "GetWeather", "type": "object", "additionalProperties": false}
```

### Tool: Function called with deserialized - fn(pydantic)

 - Deserializing {"location":"Paris, France"}
 - Expecting **10**
 - Got ↓

```
10
```



----     
## Testing ToolCollection API - fn(pydantic)

### ToolCollection: exec with serialized string - fn(pydantic)

 - Deserializing {"location":"Paris, France"} for get_weather
 - **Expecting** 10
 - **Got** ↓

```
10
```

### ToolCollection: all schemas - fn(pydantic)

```json
['Tool Name : get_weather\nTool Description : Get current temperature for a given location.\nTool Args: {"properties": {"location": {"description": "City and country e.g. San Jose, USA", "title": "Location", "type": "string"}}, "required": ["location"], "title": "GetWeather", "type": "object", "additionalProperties": false}']
```


#### Zero arg function 

This form exists for generality. No real use case for now. 

In [13]:
# Exercise 0 arg functions
# Also has no doc_string
def get_cookie():    
    return "all out!"

SERIALIZED_DATA_SAMPLE = ""
EXPECTED = "all out!"

exercise_all(test_nm="fn()", 
             test_fn=get_cookie, 
             json_str = SERIALIZED_DATA_SAMPLE, 
             expect_eval=EXPECTED)


## Testing Tool methods directly - fn()

### tool schema output - fn()

```json
{
    "type": "function",
    "function": {
        "name": "get_cookie",
        "description": "function get_cookie",
        "strict": true,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": false
        }
    }
}
```

### in-prompt schema output - fn()

```
Tool Name : get_cookie
Tool Description : function get_cookie
Tool Args: {"type": "object", "properties": {}, "required": [], "additionalProperties": false}
```

### Deserialized from string - fn()

Deserializing 
↪

⚠️ deserializer is NONE

### Function called with deserialized - fn()

 - 👉 Expecting **all out!**
 - 👉 Got **all out!** 



----     
## Testing Tool API - fn()

### Tool: tool schema output - fn()

```json
{
    "type": "function",
    "function": {
        "name": "get_cookie",
        "description": "function get_cookie",
        "strict": true,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": false
        }
    }
}
```

### Tool: in-prompt schema output - fn()

```
Tool Name : get_cookie
Tool Description : function get_cookie
Tool Args: {"type": "object", "properties": {}, "required": [], "additionalProperties": false}
```

### Tool: Function called with deserialized - fn()

 - Deserializing 
 - Expecting **all out!**
 - Got ↓

```
all out!
```



----     
## Testing ToolCollection API - fn()

### ToolCollection: exec with serialized string - fn()

 - Deserializing  for get_cookie
 - **Expecting** all out!
 - **Got** ↓

```
all out!
```

### ToolCollection: all schemas - fn()

```json
['Tool Name : get_cookie\nTool Description : function get_cookie\nTool Args: {"type": "object", "properties": {}, "required": [], "additionalProperties": false}']
```


### non-pydantic tool functions

These work similarly to the explicit pydantic version except that I do not allow specification of per-argument doc-string yet. Maybe as separate optional args later on.

In [14]:
# ⚠️⚠️👉 int|float fails. Need to program that in at some point.
def double_me(a:int):
    """
    Doubles the value of the supplied number
    """
    return str(2*a)

SERIALIZED_DATA_SAMPLE = "{\"a\" : 2}"
EXPECTED = "4"

exercise_all(test_nm="fn(a:int)", 
             test_fn=double_me, 
             json_str = SERIALIZED_DATA_SAMPLE, 
             expect_eval=EXPECTED)

12:50:47 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a)



## Testing Tool methods directly - fn(a:int)

### tool schema output - fn(a:int)

```json
{
    "type": "function",
    "function": {
        "name": "double_me",
        "description": "Doubles the value of the supplied number",
        "strict": true,
        "parameters": {
            "properties": {
                "a": {
                    "title": "A",
                    "type": "integer"
                }
            },
            "required": [
                "a"
            ],
            "title": "double_me_args",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### in-prompt schema output - fn(a:int)

```
Tool Name : double_me
Tool Description : Doubles the value of the supplied number
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}}, "required": ["a"], "title": "double_me_args", "type": "object", "additionalProperties": false}
```

### Deserialized from string - fn(a:int)

Deserializing {"a" : 2}
↪

```
a=2
```

### Function called with deserialized - fn(a:int)

 - 👉 Expecting **4**
 - 👉 Got **4** 


12:50:47 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a)



----     
## Testing Tool API - fn(a:int)

### Tool: tool schema output - fn(a:int)

```json
{
    "type": "function",
    "function": {
        "name": "double_me",
        "description": "Doubles the value of the supplied number",
        "strict": true,
        "parameters": {
            "properties": {
                "a": {
                    "title": "A",
                    "type": "integer"
                }
            },
            "required": [
                "a"
            ],
            "title": "double_me_args",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### Tool: in-prompt schema output - fn(a:int)

```
Tool Name : double_me
Tool Description : Doubles the value of the supplied number
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}}, "required": ["a"], "title": "double_me_args", "type": "object", "additionalProperties": false}
```

### Tool: Function called with deserialized - fn(a:int)

 - Deserializing {"a" : 2}
 - Expecting **4**
 - Got ↓

```
4
```


12:50:47 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a)



----     
## Testing ToolCollection API - fn(a:int)

### ToolCollection: exec with serialized string - fn(a:int)

 - Deserializing {"a" : 2} for double_me
 - **Expecting** 4
 - **Got** ↓

```
4
```

### ToolCollection: all schemas - fn(a:int)

```json
['Tool Name : double_me\nTool Description : Doubles the value of the supplied number\nTool Args: {"properties": {"a": {"title": "A", "type": "integer"}}, "required": ["a"], "title": "double_me_args", "type": "object", "additionalProperties": false}']
```


In [15]:
def multiply(a:int, b:int):
    """
    Multiplies the two supplied integers
    """
    return str(a*b)

SERIALIZED_DATA_SAMPLE = "{\"a\" : 2, \"b\":8}"
EXPECTED = "16"

exercise_all(test_nm="fn(a:int, b:int)", 
             test_fn=multiply, 
             json_str = SERIALIZED_DATA_SAMPLE,
             expect_eval=EXPECTED)

12:53:23 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a, b = des_obj.b)



## Testing Tool methods directly - fn(a:int, b:int)

### tool schema output - fn(a:int, b:int)

```json
{
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Multiplies the two supplied integers",
        "strict": true,
        "parameters": {
            "properties": {
                "a": {
                    "title": "A",
                    "type": "integer"
                },
                "b": {
                    "title": "B",
                    "type": "integer"
                }
            },
            "required": [
                "a",
                "b"
            ],
            "title": "multiply_args",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### in-prompt schema output - fn(a:int, b:int)

```
Tool Name : multiply
Tool Description : Multiplies the two supplied integers
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "multiply_args", "type": "object", "additionalProperties": false}
```

### Deserialized from string - fn(a:int, b:int)

Deserializing {"a" : 2, "b":8}
↪

```
a=2 b=8
```

### Function called with deserialized - fn(a:int, b:int)

 - 👉 Expecting **16**
 - 👉 Got **16** 


12:53:23 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a, b = des_obj.b)



----     
## Testing Tool API - fn(a:int, b:int)

### Tool: tool schema output - fn(a:int, b:int)

```json
{
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Multiplies the two supplied integers",
        "strict": true,
        "parameters": {
            "properties": {
                "a": {
                    "title": "A",
                    "type": "integer"
                },
                "b": {
                    "title": "B",
                    "type": "integer"
                }
            },
            "required": [
                "a",
                "b"
            ],
            "title": "multiply_args",
            "type": "object",
            "additionalProperties": false
        }
    }
}
```

### Tool: in-prompt schema output - fn(a:int, b:int)

```
Tool Name : multiply
Tool Description : Multiplies the two supplied integers
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "multiply_args", "type": "object", "additionalProperties": false}
```

### Tool: Function called with deserialized - fn(a:int, b:int)

 - Deserializing {"a" : 2, "b":8}
 - Expecting **16**
 - Got ↓

```
16
```


12:53:23 INFO:eval string for func calling = lambda des_obj, fn: fn(a = des_obj.a, b = des_obj.b)



----     
## Testing ToolCollection API - fn(a:int, b:int)

### ToolCollection: exec with serialized string - fn(a:int, b:int)

 - Deserializing {"a" : 2, "b":8} for multiply
 - **Expecting** 16
 - **Got** ↓

```
16
```

### ToolCollection: all schemas - fn(a:int, b:int)

```json
['Tool Name : multiply\nTool Description : Multiplies the two supplied integers\nTool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "multiply_args", "type": "object", "additionalProperties": false}']
```


# Evolution & Experiments

## ✔️ Use Pydantic to simplify schema creation

> This evolved over a few days but I did not keep track of the initial experiments. Am moving it here from another notebook just to record it as a primitive first step

What OpenAI _(and by extension, every other compatible vendor)_ needs.
 - A subset of JSON schema: _they don't spell it out, but from experience, I found that_
   - `ref` fields are not allowed. This means, schema references have to be resolved in-place. Thankfully an existing package works _(jsonref)_!
 - As much natural language description as possible
   - each function parameter to have a useful name and description
   - well thought out field names and descriptions

In PyDantic terms
 - To use the PyDantic model more easily, optionally make it a `dataclass`
 - Use `Field` to supply per-field description
 - use `jsonref.resolve_refs` to resolve any schema refs.

In [16]:
# Use the get_weather as the test case.
# This is our base use-case. 
#   - A function that takes a BaseModel 
#   - BaseModel serialized into Json
#   - BaseModel deserializes the serialized json from the LLM
#
# During development, simply hardcode the function's return
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class GetWeather(BaseModel):        
    location : str = Field(description="City and country e.g. San Jose, USA")

def get_weather(args: GetWeather) -> float:
    """
    Get current temperature for a given location.
    """
    return "10"    

In [17]:
import json
import inspect
import jsonref
from typing import TypeVar

F = TypeVar('F')

def buildToolJsonSchema(f:F) -> json:
    """
    f: A function of the form 'func(arg:BaseModel)` | `func()`
      Max of 1 argument and it should be a class deriving from PyDantic BaseModel  

    Eg.

    @dataclass
    class GetWeather(BaseModel):        
        location : str = Field(description="City and country e.g. San Jose, USA")

    def get_weather(args: GetWeather) -> float:
        '''
        Get current temperature for a given location.
        '''
        return 10    

    And use this thus:
    
        tool_schema = buildToolJsonSchema(get_weather)
    """    
    if inspect.isfunction(f):
        sig = inspect.signature(f)
        params = list(sig.parameters.items())

        if len(params) > 1:
            # more than 1 args is an error for us!
            raise TypeError(f"{str(f)} should have max of 1 arguments")
        elif len(params):            
            # 1 args case
            p_ann   = params[0][1].annotation            
            p_class = p_ann.__class__

            if "pydantic._internal._model_construction.ModelMetaclass" not in str(p_class):
                raise TypeError(f"{str(f)} must derive from Pydantic's BaseModel")

            # exec the class method to generate the json schema
            arg_json_schema = getattr(p_ann, 'model_json_schema').__call__()
            arg_json_schema = jsonref.replace_refs(arg_json_schema)

            # 👉 OpenAI enforces that "additionalProperties" : False is set!
            arg_json_schema["additionalProperties"] = False
                        
            # Now generate the schema for the full function
            # See https://platform.openai.com/docs/guides/function-calling
            func_schema = {
                "type"       : "function",
                "function"   : {
                    "name"       : f.__name__,
                    "description": f.__doc__.strip(),
                    "parameters" : arg_json_schema,
                    "strict"     : True
                }
            }            
        else:
            # 0 args case            
            # See https://platform.openai.com/docs/guides/function-calling
            func_schema = {
                "type"       : "function",
                "function"   : {
                    "name"       : f.__name__,
                    "description": f.__doc__.strip()                    
                }
            }        
        
        return func_schema
    else:
        raise TypeError(f"{str(f)} should be a python function")    


#------------------------------------------------------------------------
# Tool execution infrastructure
#------------------------------------------------------------------------
# Name -> { deserializer, executor} map
import json
from pydantic import ValidationError

class Tool:
    # tool_arg_deserializer(str) -> Obj. Expect pydantic's ValidationError
    # tool_func(obj)             -> str
    # tool_schema : json
    def __init__(self, name:str,
                 tool_arg_deserializer, 
                 tool_func,
                 tool_schema):
        self.name = name
        self.tool_arg_deserializer = tool_arg_deserializer
        self.tool_func = tool_func        
        self.tool_schema = tool_schema
    
    def exec(self, json_arg: str) -> str:
        if self.tool_arg_deserializer:
            try:
                logging.debug(f"Attempting to deserialize {json_arg} for tool: {self.name}")

                arg_obj = self.tool_arg_deserializer(json_arg)
                logging.debug(f"✔️ deserialized to {arg_obj}. Calling function")

                func_result = self.tool_func(arg_obj)
                logging.debug(f"✔️ function returned {func_result}")

                return func_result

            except ValidationError as e:
                logging.error(f"JSON string {json_arg} is not valid for {self.name}:{e}")
        else:            
            logging.debug(f"Calling no-arg function: {self.name}")            
            assert(json_arg == '{}' or json_arg is None)
            
            func_result = self.tool_func()
            logging.debug(f"✔️ function returned {func_result}")
            return func_result

class ToolCollection:
    def __init__(self):
        # {name : Tool}
        self.tool_dict = {}

    def register_tool(self, name:str, tool: Tool):
        if name in self.tool_dict:
            logging.warn(f"Tool {name} has already been registed. Overwriting!")
        
        assert(tool.tool_func)
        assert(tool.tool_schema)
        self.tool_dict[name] = tool
        
    def exec_tool(self, name: str, args: str) -> str :
        if not name in self.tool_dict:
            raise KeyError(f"Tool: {name} is not registered! Cannot call!")
        else:            
            tool = self.tool_dict[name]
            logging.debug(f"Executing tool: {tool.name}")
            return tool.exec(args)
        
    def get_schemas(self) :
        return [tool.tool_schema for tool in self.tool_dict.values()]


In [18]:
# Lowest level test. Call `buildToolJsonSchema` directly.
gw_schema = buildToolJsonSchema(get_weather)
DM.md( DM.json_fmt(gw_schema) )

```json
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "properties": {
                "location": {
                    "description": "City and country e.g. San Jose, USA",
                    "title": "Location",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ],
            "title": "GetWeather",
            "type": "object",
            "additionalProperties": false
        },
        "strict": true
    }
}
```

In [38]:
# In case we want to test this with other funcs. Use f instead of 
# directly using get_weather
f = get_weather

# Now to build out the deserializer and the executor.
# Attempt to build them as lambdas so we can supply 
# them to a wrapper Tool instance
sig = inspect.signature(f)
print(sig)

params = list(sig.parameters.items())

# Expect only 1 arg.
# It's type will be the annotation
ann = [p[1].annotation for p in params][0]

# ann Can be `Parameter.empty` if there is none
print(ann)        
print(f"Name: {ann.__name__}")
print(f"Module: {ann.__module__}")
print(f"Has `model_validate_json`: {True if hasattr(ann, 'model_validate_json') else False}")        

#--------------------------------------------------
# Build the deserialization lambda dynamically
if ann.__module__ == "__main__":
        eval_str = f"lambda json_str: {ann.__name__}.model_validate_json(json_str)"
else:
        eval_str = f"lambda json_str: {ann.__module__}.{ann.__name__}.model_validate_json(json_str)"
print(f"Eval Str : {eval_str}")
deserialize_json_str = eval(eval_str)

# test it
deserialized_obj = deserialize_json_str("{\"location\":\"Paris, France\"}")
print(deserialized_obj)

#---------------------------------------------------
# build the function calling method dynamically        
fn_call = lambda deserialized: f(deserialized)
print(fn_call(deserialized_obj))

(args: __main__.GetWeather) -> float
<class '__main__.GetWeather'>
Name: GetWeather
Module: __main__
Has `model_validate_json`: True
Eval Str : lambda json_str: GetWeather.model_validate_json(json_str)
location='Paris, France'
10


In [39]:
# Tool and ToolCollection class (update those as needed)
# with externally supplied lambdas for execution and de-serialization.
# use the variables created in the preceeding cells
gw_tool = Tool("get_weather",
                tool_arg_deserializer = deserialize_json_str, 
                tool_func = get_weather,
                tool_schema = gw_schema)

if weather := gw_tool.exec("{\"location\":\"Paris, France\"}"):
    print(weather)
    assert(weather == "10")
else:
    assert(False)

10


In [40]:
# Now do this via the ToolCollection instance
tc = ToolCollection()

# Why force user to specify name ? Set to None and take it from the tool by default but 
# allow override. Fix this.
tc.register_tool(name="get_weather", tool=gw_tool)
if weather := tc.exec_tool("get_weather", "{\"location\":\"Paris, France\"}"):
    print(weather)
    assert(weather == "10")
else:
    assert(False)

DM.md(
    DM.json_fmt(tc.get_schemas())
)

10


```json
[
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "properties": {
                    "location": {
                        "description": "City and country e.g. San Jose, USA",
                        "title": "Location",
                        "type": "string"
                    }
                },
                "required": [
                    "location"
                ],
                "title": "GetWeather",
                "type": "object",
                "additionalProperties": false
            },
            "strict": true
        }
    }
]
```



## ✔️ Improving the Tool API to take just the py function

I need to make some changes to allow a more succint signature for `ReAct` prompts. I am thinking, I can evolve the current tool signature further. 

```python
gw_tools.register_tool(
    "get_weather",
    Tool(
        name = "get_weather",
        tool_arg_deserializer = lambda json_str: GetWeather.model_validate_json(json_str),
        tool_func   = lambda gw: get_weather(gw),
        tool_schema = buildToolJsonSchema(get_weather))
)
```

to

```diff
gw_tools.register_tool(
-    "get_weather",
    Tool(
-        name = "get_weather",
        tool_fn               = fn,
-        tool_arg_deserializer = lambda json_str: GetWeather.model_validate_json(json_str),
-        tool_func             = lambda gw: get_weather(gw),
-        tool_schema           = buildToolJsonSchema(get_weather))
)
```

Ideally, even `tool_arg_deserializer = lambda json_str: GetWeather.model_validate_json(json_str),` should be gone and built automatially. Lemme see

to 

```python
gw_tools.register_tool(fn)
```

Which should now look just like LlamaIndex's version.

All done. The following scratchpad code shows how I build replacements for `tool_func`, `tool_arg_deserializer` from the function object.

```python
# Scratchpad
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class GetWeather(BaseModel):        
    location : str = Field(description="City and country e.g. San Jose, USA")

def get_weather(args: GetWeather) -> float:
    """
    Get current temperature for a given location.
    """
    retval = "10"
    logging.debug(f"get_weather called with {args}. Returning hardcoded value {retval}")
    return retval

f = get_weather

if inspect.isfunction(f):        
        sig = inspect.signature(f)
        print(sig)

        params = list(sig.parameters.items())

        # Expect only 1 arg.
        # It's type will be the annotation
        ann = [p[1].annotation for p in params][0]

        # ann Can be `Parameter.empty` if there is none
        print(ann)        
        print(f"Name: {ann.__name__}")
        print(f"Module: {ann.__module__}")
        print(f"Has `model_validate_json`: {True if hasattr(ann, 'model_validate_json') else False}")        

        #--------------------------------------------------
        # Build the deserialization lambda dynamically
        if ann.__module__ == "__main__":
                eval_str = f"lambda json_str: {ann.__name__}.model_validate_json(json_str)"
        else:
                eval_str = f"lambda json_str: {ann.__module__}.{ann.__name__}.model_validate_json(json_str)"
        print(f"Eval Str : {eval_str}")
        deserialize_json_str = eval(eval_str)
        
        # test it
        deserialized_obj = deserialize_json_str("{\"location\":\"Paris, France\"}")
        print(deserialized_obj)

        #---------------------------------------------------
        # build the function calling method dynamically        
        fn_call = lambda deserialized: f(deserialized)
        print(fn_call(deserialized_obj))
```

## ✔️ Tool API to return in-prompt schema string as well

The tool-schema used in the `tools=` field of the chat completion API takes a rather verbose format. In a ReAct _(or similar context)_ where the function call signature is embedded in the prompt itself, folk seem to be useing a different format.

In the case of `LlamaIndex` (see [LLM_LlamaIndex_Explore.ipynb](./LLM_LlamaIndex_Explore.ipynb) for the details), they use the following format

```python
def get_react_tool_descriptions(tools: Sequence[BaseTool]) -> List[str]:
    """Tool."""
    tool_descs = []
    for tool in tools:
        tool_desc = (
            f"> Tool Name: {tool.metadata.name}\n"
            f"Tool Description: {tool.metadata.description}\n"
            f"Tool Args: {tool.metadata.fn_schema_str}\n"
        )
        tool_descs.append(tool_desc)
    return tool_descs
```

For one instance, these fields are
 - function-name
 - function-description
 - fn_schema_str: stringified-json of the args

The `Tool.build_tool_call_items` returns this as the 2nd item in the tuple. Will eventually be a field in that class.

## ✔️ Tool for functions without pydantic args

I am currently only allowing functions with pydantic args. However, langChain, LlamaIndex etc all use functions with any number of args. They seem to build models dynamically from the args. Looks straightforward enough for me to do it that way as well.

Instead of 

```python

```

I want 

```python
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

tc = ToolCollection()
tc.add_tool(Tool(add))
tc.add_tool(Tool(multiply))
```

Done and refactored and tested

### Scratchpad

## ✔️ Tool for functions without pydantic args

I am currently only allowing functions with pydantic args. However, langChain, LlamaIndex etc all use functions with any number of args. They seem to build models dynamically from the args. Looks straightforward enough for me to do it that way as well.

Instead of 

```python

```

I want 

```python
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

tc = ToolCollection()
tc.add_tool(Tool(add))
tc.add_tool(Tool(multiply))
```

Done and refactored and tested

### Scratchpad

In [41]:
import inspect
import jsonref
import json
from typing import TypeVar
from pydantic import ValidationError, BaseModel, create_model

# scratchpad
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

#--------------------------------------------------------------
# scratch pad. We need to go from a plain function to 
#  - tool_schema
#  - deserializer
#  - caller
#-------------------------------------------------------------
# Manually, it would look like this for the `multiply` function
#
# from pydantic import BaseModel, Field, create_model  
# mo = create_model(
#     'multiply_args',    
#     a=(int,...),
#     b=(int,...)) 
# print(mo)
# print(json.dumps(mo.model_json_schema(), indent=4))
#--------------------------------------------------------------
def tool_from_plain_func(fn):
    sig = inspect.signature(fn)

    # Build args to send to pydantic.create_model
    dyn_model_args = ["\"multiply_args\""]
    call_args_vec  = []
    for p in sig.parameters.values():    
        # build arg to generate the model
        arg = f"{p.name}=({p.annotation.__name__}, ...)"    
        #print(arg)    
        dyn_model_args.append(arg)    

        # build arg to call fn using model fields
        # Assumes deserialized model-object will be called des_obj
        # (deserialized object)
        call_arg = f"{p.name}=des_obj.{p.name}"
        print(call_arg)
        call_args_vec.append(call_arg)            

    cmd = f"create_model({", ".join(dyn_model_args)})"
    #print(cmd)
    dyn_model = eval(cmd)

    # Schema
    print(f"Schema :\n {json.dumps(dyn_model.model_json_schema(), indent=4)}")

    # deserializer
    des = lambda jstr: dyn_model.model_validate_json(jstr)

    jstr = json.dumps({"a":1, "b":2})
    des_obj = des(jstr)
    print(f"Testing deserialization from {jstr}. Got {des_obj}")    

    # caller
    call_eval_str = f"lambda des_obj, fn: fn({", ".join(call_args_vec)})"
    print(f"Call eval string = {call_eval_str}")
    des_fn_2 = eval(call_eval_str)                   # Does not close over fn, so sending fn as arg
    des_fn   = lambda des_obj: des_fn_2(des_obj, fn) # this closes over fn
    print(inspect.signature(des_fn_2))    
    print(inspect.signature(des_fn))    
    print(f"Testing calling from deserialized object {des_obj}. Got {des_fn(des_obj)}")
    


tool_from_plain_func(multiply)

a=des_obj.a
b=des_obj.b
Schema :
 {
    "properties": {
        "a": {
            "title": "A",
            "type": "integer"
        },
        "b": {
            "title": "B",
            "type": "integer"
        }
    },
    "required": [
        "a",
        "b"
    ],
    "title": "multiply_args",
    "type": "object"
}
Testing deserialization from {"a": 1, "b": 2}. Got a=1 b=2
Call eval string = lambda des_obj, fn: fn(a=des_obj.a, b=des_obj.b)
(des_obj, fn)
(des_obj)
Testing calling from deserialized object a=1 b=2. Got 2


## ✔️ Replace deserialization eval with actual lambda

For this BaseModel case

```python
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class GetWeather(BaseModel):        
    location : str = Field(description="City and country e.g. San Jose, USA")

def get_weather(args: GetWeather) -> float:
    """
    Get current temperature for a given location.
    """
    return "10"    
```

I used to create lambas in this fashion:

```python
if p_ann.__module__ == "__main__":
    eval_str = f"lambda json_str: {p_ann.__name__}.model_validate_json(json_str)"
else:
    eval_str = f"lambda json_str: {p_ann.__module__}.{p_ann.__name__}.model_validate_json(json_str)"
```

This worked in jupyter. However, when I split these out into python modules, the lambda execution errored out with a **GetWeather: name is not known**. The module where the lambda was being run from, did not have the `GetWeather` class in scope.

So now, how do I dynamically make a call to an argument of a function based on it's signature ? 

In [42]:
#-----------------------------------------------
# Start with existing code and see if the eval can't be replaced with 
# a getattr on the annotation. Not sure how that works yet!
import inspect
import json

sig = inspect.signature(get_weather)

params = list(sig.parameters.items())
p_ann   = params[0][1].annotation            
p_class = p_ann.__class__

# Test if we can stringify json (via json.dumps) and then deserialize 
# it via the `model_validate_json` method.
des_method = getattr(p_ann, 'model_validate_json')
print(des_method)
des = lambda json_str: des_method.__call__(json_str)
des(json.dumps({"location":"San Jose, USA"}))

<bound method BaseModel.model_validate_json of <class '__main__.GetWeather'>>


GetWeather(location='San Jose, USA')

Ok. So replace 

```python
# use pydantic's model_validate_json method
if p_ann.__module__ == "__main__":
    eval_str = f"lambda json_str: {p_ann.__name__}.model_validate_json(json_str)"
else:
    eval_str = f"lambda json_str: {p_ann.__module__}.{p_ann.__name__}.model_validate_json(json_str)"

logging.debug(f"Using {eval_str} to generate the deserializer for {fn}")
func_arg_deserializer = eval(eval_str)
```

with 

```python
des_callable = getattr(p_ann, 'model_validate_json')
func_arg_deserializer = lambda json_str: des_callable.__call__(json_str)
```

## ✔️ Fix errors with schema of tool that takes no arguments

This used to work before. Wondering if there are different versions of their end-point servers. The following schema for `get_temperature()` raised a `invalid schema. additionalProperties field must be set and it should be False`. _Also, I mean, if you know what the problem is and you are telling me how to fix it, why not just fix it and go ahead? Just drop a note that you have done it. No ? You want me to fix it just so there is no error if you drop backward compatibility in the future ?_

```json
{
    "type": "function",    
    "function": {
        "name": "get_thermostat_temperature",
        "description": "Returns the current temperature setting of the thermostat.\"",
        "strict": true,
    }
}
```

Some googling around and made the following changes to make sure that the schema for no-args case also has parameters etc.

```diff
if arg_schema:
    tool_schema["function"]["parameters"] = arg_schema
+else:
+    # empty args is not accepted. Seems to vary by mood of OpenAI
+    # this works
+    # 👉 OpenAI enforces that "additionalProperties" : False is set!
+    tool_schema["function"]["parameters"] = {
+        "type" : "object",
+        "properties" : {},
+        "required"   : [],
+        "additionalProperties" : False
+    }
```            

to change the schema to 

```json
05:58:09 DEBUG:Tool : get_thermostat_temperature, Schema=
{
    "type": "function",
    "function": {
        "name": "get_thermostat_temperature",
        "description": "Returns the current temperature setting of the thermostat.\"",
        "strict": true,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": false
        }
    }
}
```

## ⬜ Tool functions that take Union types

Want to see if something like the following can be supported. Hoping it is a straigtforward extension of what I already have. 

```python
def double_me(a:int | float) -> str:
    return 2*a
```    

Currently, this is not blocking any high-level use case. Will put the work in when needed.

# Scratchpad

In [43]:
import re

s = "a : int"
m = re.search(r'(?P<name>[^:]*?)\s*:\s*(?P<type>.*?)\s*$', s)
print(m)
print(f"Extraced Name = {m.group('name')}")
print(f"Extraced Type = {m.group('type')}")

<re.Match object; span=(0, 7), match='a : int'>
Extraced Name = a
Extraced Type = int
